Σε αυτό το σημείο πρέπει να φτιαχθούν 3 χώρες, 3 εταιρείες και 3 sectors. Το σημαντικό είναι όλα αυτά να γίνουν με κάποια σχέση με τα πραγματικά δεδομένα. Παράδειγμα:
Χώρες: Γερμανία, Φινλανδία, Ελλάδα
Κλάδοι της οικονομίας: τσιμέντα, ηλεκτροπαραγωγή, χαρτί
Το μεγεθος εξαρτάται από τον κλάδο και τη χώρα
Οι τιμές εξαρτόνται μόνο από τον κλάδο.
Ο ρυθμιστής παίζει πολλές φορές το παιχνίδι και βρίσκει με binary search την σωστή τιμή για τις άδειες. (cournot και μαλακίες :Ρ)
Επιπλέον, οι συναρτήσεις, στις τελικές τους μορφές θα πρέπει να είναι σε ξεχωριστό αρχείο στο src

In [239]:
import sympy as sp, numpy as np, gurobipy as gb, pandas as pd

In [240]:
import sympy as sp
from gurobipy import Model, LinExpr, QuadExpr, GRB

# Define a function to convert SymPy expression to Gurobi expression
def sympy_to_gurobi(sympy_expr, symbol_map, model, aux_var_count=[0]):
    """
    Recursively convert a SymPy expression to a Gurobi expression, 
    handling exponentials, powers, divisions, and other complex expressions with auxiliary variables and constraints.
    
    Parameters:
        sympy_expr (sp.Expr): SymPy expression to convert.
        symbol_map (dict): Mapping from SymPy symbols to Gurobi variables.
        model (gurobipy.Model): Gurobi model to add constraints for complex expressions.
        aux_var_count (list): A list to keep track of the auxiliary variable count.
        
    Returns:
        Gurobi expression (LinExpr, QuadExpr, or constant).
    """
    if isinstance(sympy_expr, sp.Symbol):
        return symbol_map[sympy_expr]
    
    elif isinstance(sympy_expr, sp.Add):
        return sum(sympy_to_gurobi(arg, symbol_map, model, aux_var_count) for arg in sympy_expr.args)
    
    elif isinstance(sympy_expr, sp.Mul):
        result = 1
        for arg in sympy_expr.args:
            result *= sympy_to_gurobi(arg, symbol_map, model, aux_var_count)
        return result
    
    elif isinstance(sympy_expr, sp.Pow):
        base, exp = sympy_expr.args
        
        # Always create an auxiliary variable for the base
        base_expr = sympy_to_gurobi(base, symbol_map, model, aux_var_count)
        aux_var_name = f"pow_base_aux_{aux_var_count[0]}"
        aux_var_count[0] += 1
        base_aux_var = model.addVar(name=aux_var_name, vtype=GRB.CONTINUOUS)
        model.addConstr(base_aux_var == base_expr)
        
        if exp == 2:
            # Handle quadratic expression
            return QuadExpr(base_aux_var * base_aux_var)
        else:
            # Handle non-quadratic powers using general constraints
            exp_value = float(exp)
            aux_var_name = f"pow_aux_{aux_var_count[0]}"
            aux_var_count[0] += 1
            pow_aux_var = model.addVar(name=aux_var_name, vtype=GRB.CONTINUOUS)
            model.addGenConstrPow(base_aux_var, pow_aux_var, exp_value)
            return pow_aux_var
    
    elif isinstance(sympy_expr, sp.exp):
        arg_expr = sympy_to_gurobi(sympy_expr.args[0], symbol_map, model, aux_var_count)
        aux_var_name = f"exp_aux_{aux_var_count[0]}"
        aux_var_count[0] += 1
        arg_aux_var = model.addVar(name=f"aux_{aux_var_name}_arg", lb=0, vtype=GRB.CONTINUOUS)
        model.addConstr(arg_aux_var == arg_expr)
        exp_aux_var = model.addVar(name=aux_var_name, vtype=GRB.CONTINUOUS)
        model.addGenConstrExp(arg_aux_var, exp_aux_var)
        return exp_aux_var
    
    elif isinstance(sympy_expr, sp.log):
        arg_expr = sympy_to_gurobi(sympy_expr.args[0], symbol_map, model, aux_var_count)
        aux_var_name = f"log_aux_{aux_var_count[0]}"
        aux_var_count[0] += 1
        arg_aux_var = model.addVar(name=f"aux_{aux_var_name}_arg", vtype=GRB.CONTINUOUS)
        model.addConstr(arg_aux_var == arg_expr)
        log_aux_var = model.addVar(name=aux_var_name, vtype=GRB.CONTINUOUS)
        model.addGenConstrLog(arg_aux_var, log_aux_var)
        return log_aux_var

    elif isinstance(sympy_expr, sp.sin):
        arg_expr = sympy_to_gurobi(sympy_expr.args[0], symbol_map, model, aux_var_count)
        aux_var_name = f"sin_aux_{aux_var_count[0]}"
        aux_var_count[0] += 1
        arg_aux_var = model.addVar(name=f"aux_{aux_var_name}_arg", vtype=GRB.CONTINUOUS)
        model.addConstr(arg_aux_var == arg_expr)
        sin_aux_var = model.addVar(name=aux_var_name, vtype=GRB.CONTINUOUS)
        # Add piecewise constraints here for approximation
        return sin_aux_var

    elif isinstance(sympy_expr, sp.cos):
        arg_expr = sympy_to_gurobi(sympy_expr.args[0], symbol_map, model, aux_var_count)
        aux_var_name = f"cos_aux_{aux_var_count[0]}"
        aux_var_count[0] += 1
        arg_aux_var = model.addVar(name=f"aux_{aux_var_name}_arg", vtype=GRB.CONTINUOUS)
        model.addConstr(arg_aux_var == arg_expr)
        cos_aux_var = model.addVar(name=aux_var_name, vtype=GRB.CONTINUOUS)
        # Add piecewise constraints here for approximation
        return cos_aux_var

    elif isinstance(sympy_expr, sp.tan):
        arg_expr = sympy_to_gurobi(sympy_expr.args[0], symbol_map, model, aux_var_count)
        aux_var_name = f"tan_aux_{aux_var_count[0]}"
        aux_var_count[0] += 1
        arg_aux_var = model.addVar(name=f"aux_{aux_var_name}_arg", vtype=GRB.CONTINUOUS)
        model.addConstr(arg_aux_var == arg_expr)
        tan_aux_var = model.addVar(name=aux_var_name, vtype=GRB.CONTINUOUS)
        # Add piecewise constraints here for approximation
        return tan_aux_var

    elif isinstance(sympy_expr, sp.Mul) and any(isinstance(arg, sp.Pow) and arg.exp == -1 for arg in sympy_expr.args):
        # Handling division by separating numerator and denominator
        numerator = 1
        denominator = 1
        for arg in sympy_expr.args:
            if isinstance(arg, sp.Pow) and arg.exp == -1:
                denominator *= arg.base
            else:
                numerator *= arg
        
        # Handle numerator
        num_expr = sympy_to_gurobi(numerator, symbol_map, model, aux_var_count)
        num_aux_var_name = f"num_aux_{aux_var_count[0]}"
        aux_var_count[0] += 1
        num_aux_var = model.addVar(name=num_aux_var_name, vtype=GRB.CONTINUOUS)
        model.addConstr(num_aux_var == num_expr)
        
        # Handle denominator
        denom_expr = sympy_to_gurobi(denominator, symbol_map, model, aux_var_count)
        denom_aux_var_name = f"denom_aux_{aux_var_count[0]}"
        aux_var_count[0] += 1
        denom_aux_var = model.addVar(name=denom_aux_var_name, vtype=GRB.CONTINUOUS)
        model.addConstr(denom_aux_var == denom_expr)
        
        # Create the auxiliary variable for the inverse of the denominator
        inv_denom_aux_var_name = f"inv_denom_aux_{aux_var_count[0]}"
        aux_var_count[0] += 1
        inv_denom_aux_var = model.addVar(name=inv_denom_aux_var_name, vtype=GRB.CONTINUOUS)
        model.addConstr(inv_denom_aux_var * denom_aux_var == 1)
        
        # Final expression: numerator * (1/denominator)
        return num_aux_var * inv_denom_aux_var

    elif isinstance(sympy_expr, sp.Number):
        return float(sympy_expr)
    
    else:
        raise ValueError(f"Unsupported SymPy expression: {sympy_expr}")

In [241]:

#Generate one regulator
x = sp.symbols('x')

class Regulator:
    _id_counter = 1

    def __init__(self, name, permit_price, emission_cap):
        self.id = Regulator._id_counter
        Regulator._id_counter += 1
        self.name = name
        self.permit_price = permit_price
        self.emission_cap = emission_cap

    def __repr__(self):
        return f"Regulator(id={self.id}, name='{self.name}', permit_price={self.permit_price}, emission_cap={self.emission_cap})"

# Global registries for sectors and countries
sector_registry = {}
country_registry = {}
firm_registry = {}
class Sector:
    _id_counter = 1

    def __init__(self, name, price_demand_function, free_emission_multiplier):
        self.id = Sector._id_counter
        Sector._id_counter += 1
        self.name = name
        self.price_demand_function = price_demand_function
        self.free_emission_multiplier = free_emission_multiplier
        self.firms = []  # List to store firms in this sector

        # Register this sector in the global registry
        sector_registry[self.id] = self

    def add_firm(self, firm):
        self.firms.append(firm)

    def __repr__(self):
        return f"Sector(id={self.id}, name='{self.name}', firms={len(self.firms)})"

class Country:
    _id_counter = 1

    def __init__(self, name, size):
        self.id = Country._id_counter
        Country._id_counter += 1
        self.name = name
        self.size = size
        self.firms = []  # List to store firms in this country

        # Register this country in the global registry
        country_registry[self.id] = self

    def add_firm(self, firm):
        self.firms.append(firm)

    def __repr__(self):
        return f"Country(id={self.id}, name='{self.name}', firms={len(self.firms)})"

class Firm:
    _id_counter = 1

    def __init__(self, name, sector, country, production_cost_function, abatement_cost_function, actual_output, emission, profit):
        self.id = Firm._id_counter
        Firm._id_counter += 1
        self.name = name

        # Determine sector and country from either object or ID
        if isinstance(sector, Sector):
            self.sector = sector
        elif isinstance(sector, int):
            self.sector = sector_registry.get(sector)

        if isinstance(country, Country):
            self.country = country
        elif isinstance(country, int):
            self.country = country_registry.get(country)

        self.production_cost_function = production_cost_function
        self.abatement_cost_function = abatement_cost_function
        self.actual_output = actual_output
        self.emission = emission
        self.profit = profit

        # Register this firm in the global registry
        firm_registry[self.id] = self
        
        # Register this firm with its sector and country
        if self.sector:
            self.sector.add_firm(self)
        if self.country:
            self.country.add_firm(self)

    def __repr__(self):
        return f"Firm(id={self.id}, name='{self.name}', sector_id={self.sector.id if self.sector else None}, country_id={self.country.id if self.country else None}, actual_output={self.actual_output}, emission={self.emission}, profit={self.profit})"
    


In [242]:
# Example Usage
import sympy as sp

# Create Sectors and Countries
Regulator1 = Regulator('EU', permit_price= 2, emission_cap= 500)
sector1 = Sector('cement', price_demand_function= 10 - 0.1*sp.symbols('x'), free_emission_multiplier= 0.1)
sector2 = Sector('steel', price_demand_function=10 - 0.1*sp.symbols('x'), free_emission_multiplier= 0.02)
sector3 = Sector('paper', price_demand_function= 200 - 0.02*sp.symbols('x')**1.5, free_emission_multiplier= 0.1)
country1 = Country('DE', 1)
country2 = Country('FI', 0.5)
country3 = Country('GR', size= 0.1)

# Create Firms using objects
firm1 = Firm('firm1', 1, 1, sp.symbols('x')*0, 5*sp.symbols('x')**2, 0, 0, 0)
firm2 = Firm('firm2', 1, 1, sp.symbols('x')*0, 5*sp.symbols('x')**2, 0, 0, 0)
firm3 = Firm('firm3', 1, 3, sp.symbols('x')*0, sp.symbols('x')**1.5, 0, 0, 0)
firm4 = Firm('firm4', 2, 1, sp.symbols('x')*0, sp.symbols('x')**1.1, 0, 0, 0)
firm5 = Firm('firm5', 2, 2, sp.symbols('x')*0, sp.symbols('x')**1.2, 0, 0, 0)
firm6 = Firm('firm6', 2, 3, sp.symbols('x')*0, sp.symbols('x')**20, 0, 0, 0)
firm7 = Firm('firm7', 3, 1, sp.symbols('x')*0, sp.symbols('x')**5, 0, 0, 0)
firm8 = Firm('firm8', 3, 2, sp.symbols('x')*0, sp.symbols('x')**3, 0, 0, 0)
firm9 = Firm('firm9', 3, 3, sp.symbols('x')*0, sp.symbols('x')**10, 0, 0, 0)


print(sector1.firms)  # List of firms in sector1
print(country1.firms)  # List of firms in country1
print(firm_registry)  # Dictionary of all firms

for firm in firm_registry.values():
    print(firm.name, firm.sector.name, firm.country.name)

[Firm(id=1, name='firm1', sector_id=1, country_id=1, actual_output=0, emission=0, profit=0), Firm(id=2, name='firm2', sector_id=1, country_id=1, actual_output=0, emission=0, profit=0), Firm(id=3, name='firm3', sector_id=1, country_id=3, actual_output=0, emission=0, profit=0)]
[Firm(id=1, name='firm1', sector_id=1, country_id=1, actual_output=0, emission=0, profit=0), Firm(id=2, name='firm2', sector_id=1, country_id=1, actual_output=0, emission=0, profit=0), Firm(id=4, name='firm4', sector_id=2, country_id=1, actual_output=0, emission=0, profit=0), Firm(id=7, name='firm7', sector_id=3, country_id=1, actual_output=0, emission=0, profit=0)]
{1: Firm(id=1, name='firm1', sector_id=1, country_id=1, actual_output=0, emission=0, profit=0), 2: Firm(id=2, name='firm2', sector_id=1, country_id=1, actual_output=0, emission=0, profit=0), 3: Firm(id=3, name='firm3', sector_id=1, country_id=3, actual_output=0, emission=0, profit=0), 4: Firm(id=4, name='firm4', sector_id=2, country_id=1, actual_output

In [243]:
def calculate_output(firm, firms, regulator, dummy_sector, verbose=False, writeLP=0):
    # Calculate the output of the firm

    # Sum of all other outputs
    sum_other_outputs = 0
    for i in range(len(dummy_sector.firms)):
        if dummy_sector.firms[i].id != firm.id:
            sum_other_outputs += dummy_sector.firms[i].actual_output
    
    #Get the price of the permits
    permit_price = regulator.permit_price
    # Get the price demand function of the sector
    price_demand_function = dummy_sector.price_demand_function
    # Get the abatement cost function of the firm
    abatement_cost_function = firm.abatement_cost_function
    # Get the production cost function of the firm
    production_cost_function = firm.production_cost_function
    # Get the free emission multiplier of the firm
    free_emission_multiplier = dummy_sector.free_emission_multiplier


    out, em = sp.symbols('out em')


    # Calculate the output of the firm


    #print("Price to demand Function: {}".format(price_demand_function.subs(x, sum_other_outputs + out)))
    income = (price_demand_function.subs(x, sum_other_outputs + out) - production_cost_function.subs(x, out))*out
    abatement = -abatement_cost_function.subs(x, out - em)
    trading = - permit_price * (em -free_emission_multiplier * out)
    profit_expr = income + abatement + trading
    # print("Profit Expression: {}".format(profit_expr))
    # Solve the optimization problem
    m = gb.Model("firm")
    output = m.addVar(vtype = gb.GRB.CONTINUOUS, name = "output", lb = 0)
    emission = m.addVar(vtype = gb.GRB.CONTINUOUS, name = "emission", lb = 0)
    symbol_map = {out: output, em: emission}
    profit = sympy_to_gurobi(profit_expr, symbol_map, m)
    m.setObjective(profit, gb.GRB.MAXIMIZE) 
    m.addConstr(emission <= output)

    m.params.OutputFlag = 1 if verbose else 0

    m.optimize()
    if writeLP>0:
        m.write("firm{}.lp".format(writeLP))
    return output.X, emission.X, profit.getValue()

# Example Usage
output, emission, profit = calculate_output(firm1, firm_registry.values(), Regulator1, sector1)
print(f"Output: {output}, Emission: {emission}, Profit: {profit}")

Output: 41.00000000007049, Emission: 40.799999984144065, Profit: 168.300000000001


In [244]:
import math
import sys

def optimize_them_all(firms, dummy_market, print_output=True,print_diff=False, presicion = 0.01):
    repeat = True
    
    while repeat:
        lp_counter = 0
        max_diff = 0
        repeat = False
        for sector in sector_registry.values():
            for firm in sector.firms:
                output, emission, profit = calculate_output(firm, firms, dummy_market, firm.sector , verbose=False, writeLP=lp_counter)
                # lp_counter += 1
                if abs(output - firm.actual_output)>presicion or abs(emission - firm.emission)>presicion:
                    repeat = True
                max_diff = max(max_diff, abs(output - firm.actual_output), abs(emission - firm.emission))
                firm.actual_output = output
                firm.emission = emission
                firm.profit = profit
                if(print_output):
                    print("Firm {} has output: {:5f} and emission: {:5f} and profit: {:2f}".format(firm.name, firm.actual_output, firm.emission, profit))
        if(print_diff): 
            sys.stdout.write("\rMax diff: {:5f}".format(max_diff))
            sys.stdout.flush()


optimize_them_all(list(firm_registry.values()), Regulator1, print_output=False, print_diff=True)


Max diff: 0.00830704

In [245]:
for i in firm_registry.values():
    print(i.name, i.actual_output, i.emission, i.profit)

firm1 20.50008332350278 20.300083305705307 42.22534162642652
firm2 20.4999746687974 20.299974651006234 42.224896141491705
firm3 20.49997100384991 18.730364472453758 43.210047507786946
firm4 23.824858660036572 0.0 59.83873476687163
firm5 18.858384859071684 7.034610134492777 39.833487701656374
firm6 18.858378240445866 17.969981194426587 37.24685019197733
firm7 117.89988476313636 117.09511917320486 7678.211989516225
firm8 117.91325878606912 117.09276697773944 7679.486794786495
firm9 117.90235275129926 117.0665215850784 7678.740736232599


In [246]:
def get_emission(firms):
    total_emission = 0
    for firm in firms:
        total_emission += firm.emission
    return total_emission

def regulator_problem(firms, regulator, presicion = 0.1):
    repeat = True
    
    while repeat:
        print("Permit price: {}".format(regulator.permit_price))
        repeat = False
        total_emission = get_emission(firms)
        if total_emission - regulator.emission_cap > 10:
            regulator.permit_price += 1
            optimize_them_all(firms, regulator, print_output=False, print_diff=True, presicion = presicion)
            repeat = True
        elif - total_emission + regulator.emission_cap > 10:
            regulator.permit_price -= 1
            optimize_them_all(firms, regulator, print_output=False, print_diff=True, presicion = presicion)
            repeat = True
        else:
            break
    return regulator.permit_price
Regulator1.emission_cap =  regulator_problem(list(firm_registry.values()), Regulator1)
print(Regulator1.emission_cap)
for i in firm_registry.values():
    print(i.name, i.actual_output, i.emission, i.profit)

Permit price: 2
Max diff: 0.0820165Permit price: 1
Max diff: 0.091248Permit price: 0
0
firm1 24.97473751406455 24.974711503065475 62.373751385922596
firm2 25.02458044906558 25.024554443885094 62.62296266145597
firm3 25.00034101843493 25.00034101843493 62.501705103804014
firm4 24.972495308468314 24.972495308468314 62.362552193147195
firm5 25.026688792390754 25.026688792390754 62.6335151911177
firm6 25.000407949570462 25.000407949570462 62.5020397644946
firm7 118.58542269773118 118.58542269773118 7790.6353510199915
firm8 118.60332738648432 118.60232738648432 7792.534497060051
firm9 118.73726878905799 118.73726878905799 7797.184510322986
